# Unit 5 - Financial Planning


In [1]:
!pip install alpaca-trade-api

In [2]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
import numpy as np
from datetime import datetime as dt


%matplotlib inline

In [3]:
# Load .env enviroment variables
load_dotenv()

True

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [4]:
# Set current amount of crypto assets
nomics_api = os.getenv('NOMICS')
crypto = requests.get('https://api.nomics.com/v1/currencies/ticker?key='+ nomics_api +'&ids=BTC,ETH,XRP&interval=1d,30d&convert=EUR&per-page=100&page=1').json()
crypto

[{'id': 'BTC',
  'currency': 'BTC',
  'symbol': 'BTC',
  'name': 'Bitcoin',
  'logo_url': 'https://s3.us-east-2.amazonaws.com/nomics-api/static/images/currencies/btc.svg',
  'status': 'active',
  'price': '14692.99659919',
  'price_date': '2020-12-11T00:00:00Z',
  'price_timestamp': '2020-12-11T03:11:00Z',
  'circulating_supply': '18567731',
  'max_supply': '21000000',
  'market_cap': '272815608438',
  'num_exchanges': '372',
  'num_pairs': '46514',
  'num_pairs_unmapped': '5584',
  'first_candle': '2011-08-18T00:00:00Z',
  'first_trade': '2011-08-18T00:00:00Z',
  'first_order_book': '2017-01-06T00:00:00Z',
  'rank': '1',
  'rank_delta': '0',
  'high': '16451.53653244',
  'high_timestamp': '2020-11-30T00:00:00Z',
  '1d': {'volume': '21681702664.32',
   'price_change': '-554.82790367',
   'price_change_pct': '-0.0364',
   'volume_change': '-3021591800.77',
   'volume_change_pct': '-0.1223',
   'market_cap_change': '-10289117589.70',
   'market_cap_change_pct': '-0.0363'},
  '30d': {'vol

In [5]:
btc_price = crypto[0]['price']
eth_price = crypto[1]['price']
xrp_price = crypto[2]['price']
print(btc_price, eth_price, xrp_price)
d = {'BTC': [float(btc_price)], 'ETH': [float(eth_price)], 'XRP': [float(xrp_price)]}
crypto_data = pd.DataFrame(data=d)
crypto_data

14692.99659919 447.71021959 0.45738258


,BTC,ETH,XRP
0,14692.996599,447.71022,0.457383


In [6]:
# Set Alpaca API key and secret
alpaca_api = os.getenv('ALPACA_API_KEY')
alpaca_secret = os.getenv('ALPACA_SECRET_KEY')

# Create the Alpaca API object
api = tradeapi.REST(alpaca_api, alpaca_secret, api_version='v2')

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

C:\Users\murip\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  after removing the cwd from sys.path.


In [8]:
# Format current date as ISO format
current_date = pd.Timestamp('2020-12-10', tz='America/New_York').isoformat()

# Set the tickers
tickers = ["KO", "PFE", 'O', 'GILD', 'D', 'MAIN', 'UBER', 'PINS', 'WKHS', 'CTSH']

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for stocks
ticker_data = api.get_barset(tickers, timeframe, start=current_date).df

# Preview DataFrame
ticker_data.head()

CTSH                                    D        \
                            open   high    low  close   volume   open  high   
2020-12-10 00:00:00-05:00  79.01  79.01  77.94  78.38  1646800  75.01  75.3   

                                                   GILD                        \
                             low  close   volume   open    high    low  close   
2020-12-10 00:00:00-05:00  73.96  74.34  4346470  60.66  60.895  60.13  60.43   

                                      KO                                 MAIN  \
                            volume  open   high    low  close   volume   open   
2020-12-10 00:00:00-05:00  6183380  53.4  53.51  52.98  53.05  6867934  31.32   

                                                           O                \
                           high    low  close  volume   open   high    low   
2020-12-10 00:00:00-05:00  31.5  30.88  31.49  412237  60.41  60.62  59.84   

                                             PFE                        \
                           close   volume   open   high     low  close   
2020-12-10 00:00:00-05:00  59.94  2530467  41.82  42.22  41.445  41.73   

                                      PINS                               \
                             volume   open   high   low  close   volume   
2020-12-10 00:00:00-05:00  47842903  67.78  71.91  66.6  71.68  7900624   

                            UBER                                 WKHS         \
                            open   high   low  close    volume   open   high   
2020-12-10 00:00:00-05:00  52.33  54.62  51.3  54.38  18241728  21.05  21.96   

                                                    
                             low    close   volume  
2020-12-10 00:00:00-05:00  20.65  21.1485  8285186

In [9]:
# Pick close prices
new_data = pd.concat([ticker_data['KO']['close'], ticker_data['PFE']['close'], ticker_data['O']['close'], ticker_data['GILD']['close'], ticker_data['D']['close'], ticker_data['MAIN']['close'],
                     ticker_data['UBER']['close'], ticker_data['PINS']['close'], ticker_data['WKHS']['close'], ticker_data['CTSH']['close']], 
                    axis=1)
new_data

,close,close,close,close,close,close,close,close,close,close
2020-12-10 00:00:00-05:00,53.05,41.73,59.94,60.43,74.34,31.49,54.38,71.68,21.1485,78.38


In [10]:
new_data.columns = ['KO', 'PFE', 'O', 'GILD', 'D', 'MAIN', 'UBER', 'PINS', 'WKHS', 'CTSH']
new_data

,KO,PFE,O,GILD,D,MAIN,UBER,PINS,WKHS,CTSH
2020-12-10 00:00:00-05:00,53.05,41.73,59.94,60.43,74.34,31.49,54.38,71.68,21.1485,78.38


In [11]:
# Compute the current value of shares
bonds_investment = 10000
crypto_investment = 20000
stocks_investment = 20000


In [12]:
# Setting bonds
bond_tickers = ['TLT', 'BLV', 'IGLB']

# Get current closing prices for stocks
ticker_bond_data = api.get_barset(bond_tickers, timeframe, start=current_date).df

# Preview DataFrame
ticker_bond_data.head()

BLV                                   IGLB  \
                             open    high     low   close  volume   open   
2020-12-10 00:00:00-05:00  111.61  112.38  111.61  112.28  162614  72.03   

                                                           TLT          \
                           high    low   close  volume    open    high   
2020-12-10 00:00:00-05:00  72.5  72.02  72.425  401638  157.24  158.29   

                                                    
                              low   close   volume  
2020-12-10 00:00:00-05:00  156.83  158.21  8366161

In [13]:
# Selecting closing price for bonds
close_bonds = pd.concat([ticker_bond_data['BLV']['close'], ticker_bond_data['IGLB']['close'], ticker_bond_data['TLT']['close']], axis=1)
close_bonds.columns = ['BLV', 'IGLB', 'TLT']
close_bonds

,BLV,IGLB,TLT
2020-12-10 00:00:00-05:00,112.28,72.425,158.21


In [14]:
# Get current closing prices for stocks
sp500_data = pd.read_csv('sp500_history.csv', index_col='Date', parse_dates=True, infer_datetime_format=True)

# Preview DataFrame
sp500_data.head()

,Close
Date,
2019-04-23,$2933.68
2019-04-22,$2907.97
2019-04-18,$2905.03
2019-04-17,$2900.45
2019-04-16,$2907.06


In [15]:
investment_per_stock = round(stocks_investment / len(new_data.columns),2)
investment_per_stock

investment_per_crypto = round(crypto_investment / len(crypto_data.columns),2)
investment_per_crypto

investment_per_bond = round(bonds_investment / len(close_bonds.columns),2)
investment_per_crypto

print(investment_per_stock, investment_per_crypto, investment_per_crypto)

2000.0 6666.67 6666.67


In [16]:
# Get stock value, equal investments
stock_value = new_data * investment_per_stock
stock_value

,KO,PFE,O,GILD,D,MAIN,UBER,PINS,WKHS,CTSH
2020-12-10 00:00:00-05:00,106100.0,83460.0,119880.0,120860.0,148680.0,62980.0,108760.0,143360.0,42297.0,156760.0


In [17]:
# Get crypto value, equal investment
crypto_value = crypto_data * crypto_data
crypto_value

,BTC,ETH,XRP
0,2.158841e+08,200444.440725,0.209199


# Sharpe ratios

In [18]:
# Historical crypto data
current_date = pd.Timestamp('2015-12-10', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-12-10', tz='America/New_York').isoformat()

btc_data = pd.read_csv('BTC.csv', parse_dates=True, infer_datetime_format=True)
eth_data = pd.read_csv('ETH.csv', parse_dates=True, infer_datetime_format=True)
xrp_data = pd.read_csv('XRP.csv', parse_dates=True, infer_datetime_format=True)

btc_data = btc_data.set_index('Date')
btc_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


In [19]:
eth_data = eth_data.set_index('Date')
eth_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-08-07,2.831620,3.536610,2.521120,2.772120,2.772120,164329
2015-08-08,2.793760,2.798810,0.714725,0.753325,0.753325,674188
2015-08-09,0.706136,0.879810,0.629191,0.701897,0.701897,532170
2015-08-10,0.713989,0.729854,0.636546,0.708448,0.708448,405283
2015-08-11,0.708087,1.131410,0.663235,1.067860,1.067860,1463100


In [20]:
xrp_data = xrp_data.set_index('Date')
xrp_data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,0.005123,0.005803,0.005123,0.005399,0.005399,1281960
2014-09-18,0.005398,0.005521,0.005091,0.005135,0.005135,491147
2014-09-19,0.005134,0.005167,0.004873,0.004942,0.004942,407109
2014-09-20,0.004942,0.004954,0.004736,0.004772,0.004772,322621
2014-09-21,0.004768,0.004791,0.004540,0.004570,0.004570,309571


## Get pct change only

In [21]:
all_crypto = pd.concat([btc_data['Adj Close'], eth_data['Adj Close'], xrp_data['Adj Close']],
                      axis=1, join='inner')
all_crypto.columns = ['BTC Adj Close', 'ETH Adj Close', 'XRP Adj Close']
all_crypto.head()

,BTC Adj Close,ETH Adj Close,XRP Adj Close
Date,,,
2015-08-07,279.584991,2.772120,0.008152
2015-08-08,260.997009,0.753325,0.008476
2015-08-09,265.083008,0.701897,0.008808
2015-08-10,264.470001,0.708448,0.008750
2015-08-11,270.385986,1.067860,0.008591


In [22]:
crypto_pct_change = all_crypto.pct_change().dropna()
crypto_pct_change.head()

,BTC Adj Close,ETH Adj Close,XRP Adj Close
Date,,,
2015-08-08,-0.066484,-0.728249,0.039745
2015-08-09,0.015655,-0.068268,0.039169
2015-08-10,-0.002313,0.009333,-0.006585
2015-08-11,0.022369,0.507323,-0.018171
2015-08-12,-0.014831,0.140075,-0.037248


In [23]:
# Historical bonds data
# Set the tickers
tickers_bonds = ['BLV', 'IGLB', 'TLT']

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

bonds_historical = api.get_barset(tickers_bonds, timeframe, start=current_date, end=end_date,).df
bonds_historical.head()

BLV                                      IGLB  \
                            open     high      low  close    volume   open   
2015-12-10 00:00:00-05:00  88.27  88.4400  88.0701  88.28  155466.0  56.62   
2015-12-11 00:00:00-05:00  88.75  88.8600  88.4000  88.83  160535.0  56.72   
2015-12-14 00:00:00-05:00  88.44  88.6000  87.3300  87.67  307403.0  56.32   
2015-12-15 00:00:00-05:00  87.32  87.7700  87.2700  87.73  145482.0  55.79   
2015-12-16 00:00:00-05:00  87.50  88.1799  87.0700  87.60  54229.0   55.90   

                                                                   TLT  \
                              high      low    close    volume    open   
2015-12-10 00:00:00-05:00  56.7099  56.4600  56.4800  113110.0  121.79   
2015-12-11 00:00:00-05:00  56.7368  56.4500  56.5000  82668.0   123.05   
2015-12-14 00:00:00-05:00  56.4800  55.8500  55.8701  289998.0  123.08   
2015-12-15 00:00:00-05:00  56.0500  55.7600  55.9001  304139.0  121.02   
2015-12-16 00:00:00-05:00  56.1600  55.7101  55.8700  112087.0  121.07   

                                                              
                             high     low   close     volume  
2015-12-10 00:00:00-05:00  122.03  121.49  121.82  3612478.0  
2015-12-11 00:00:00-05:00  124.10  122.87  123.75  9357518.0  
2015-12-14 00:00:00-05:00  123.15  121.75  122.14  8363553.0  
2015-12-15 00:00:00-05:00  121.60  120.83  121.48  6511469.0  
2015-12-16 00:00:00-05:00  122.17  120.50  121.17  9083300.0

In [24]:
"""
# to move index into column from dataframe
df = df.reset_index()

#if date/index column is called time
df['time'] = df['time'].dt.tz_localize(None)
"""

"\n# to move index into column from dataframe\ndf = df.reset_index()\n\n#if date/index column is called time\ndf['time'] = df['time'].dt.tz_localize(None)\n"

In [25]:
# Select only close prices for bonds
bonds_close_only = pd.concat([bonds_historical['BLV']['close'], bonds_historical['IGLB']['close'], bonds_historical['TLT']['close']], 
                    axis=1)
bonds_close_only.columns = ['BLV', 'IGLB', 'TLT']
bonds_close_only.head()

,BLV,IGLB,TLT
2015-12-10 00:00:00-05:00,88.28,56.4800,121.82
2015-12-11 00:00:00-05:00,88.83,56.5000,123.75
2015-12-14 00:00:00-05:00,87.67,55.8701,122.14
2015-12-15 00:00:00-05:00,87.73,55.9001,121.48
2015-12-16 00:00:00-05:00,87.60,55.8700,121.17


In [30]:
bonds_returns = bonds_close_only.pct_change().dropna()
bonds_returns.head()

,BLV,IGLB,TLT
2015-12-11 00:00:00-05:00,0.006230,0.000354,0.015843
2015-12-14 00:00:00-05:00,-0.013059,-0.011149,-0.013010
2015-12-15 00:00:00-05:00,0.000684,0.000537,-0.005404
2015-12-16 00:00:00-05:00,-0.001482,-0.000538,-0.002552
2015-12-17 00:00:00-05:00,0.006050,0.006266,0.011471


In [34]:
# Removing time from DateTime
bonds_returns.index = pd.to_datetime(bonds_returns.index).date
bonds_returns.head()

,BLV,IGLB,TLT
2015-12-11,0.006230,0.000354,0.015843
2015-12-14,-0.013059,-0.011149,-0.013010
2015-12-15,0.000684,0.000537,-0.005404
2015-12-16,-0.001482,-0.000538,-0.002552
2015-12-17,0.006050,0.006266,0.011471


In [36]:
# Historical stock data
# Set the tickers
tickers = ["KO", "PFE", 'O', 'GILD', 'D', 'MAIN', 'UBER', 'PINS', 'WKHS', 'CTSH']

# Set timeframe to '1D' for Alpaca API

stock_historical = api.get_barset(tickers, timeframe, start=current_date, end=end_date,).df

stock_historical.head()

CTSH                                      D  \
                            open   high    low  close     volume   open   
2015-12-10 00:00:00-05:00  59.63  60.06  59.27  59.52  2880954.0  66.13   
2015-12-11 00:00:00-05:00  58.81  59.13  57.78  58.29  4408789.0  64.92   
2015-12-14 00:00:00-05:00  58.89  59.77  58.67  59.75  5236838.0  64.81   
2015-12-15 00:00:00-05:00  60.34  61.05  59.59  59.66  4464072.0  65.60   
2015-12-16 00:00:00-05:00  60.05  62.52  59.95  62.41  5265173.0  66.50   

                                                            GILD           \
                             high    low  close   volume    open     high   
2015-12-10 00:00:00-05:00  66.240  65.14  65.20  1559244  102.45  103.060   
2015-12-11 00:00:00-05:00  65.444  64.54  64.89  1666716  101.39  102.465   
2015-12-14 00:00:00-05:00  65.440  64.55  65.42  2029797  99.94   100.930   
2015-12-15 00:00:00-05:00  66.695  65.42  66.15  1739872  101.74  103.000   
2015-12-16 00:00:00-05:00  67.630  66.31  67.57  2559933  103.18  103.490   

                                                       KO                 \
                              low   close   volume   open   high     low   
2015-12-10 00:00:00-05:00  101.56  102.63  6273977  42.73  43.07  42.525   
2015-12-11 00:00:00-05:00  99.67   100.02  7656568  42.63  42.83  42.140   
2015-12-14 00:00:00-05:00  98.66   100.50  7716919  42.31  42.76  41.970   
2015-12-15 00:00:00-05:00  101.44  102.02  8995081  42.84  43.24  42.760   
2015-12-16 00:00:00-05:00  101.64  103.32  6391011  43.25  43.91  43.250   

                                             MAIN                           \
                           close    volume   open     high    low    close   
2015-12-10 00:00:00-05:00  42.74  12983602  30.14  31.0200  29.86  30.9200   
2015-12-11 00:00:00-05:00  42.28  14013678  30.62  30.9100  29.92  30.1200   
2015-12-14 00:00:00-05:00  42.43  17901923  30.20  30.4600  28.74  29.0799   
2015-12-15 00:00:00-05:00  43.07  15696021  28.76  29.4099  28.52  29.0300   
2015-12-16 00:00:00-05:00  43.83  19790291  29.23  29.3800  28.70  29.2600   

                                         O                                     \
                             volume   open     high    low   close     volume   
2015-12-10 00:00:00-05:00  337722.0  49.57  49.9200  48.92  48.950  1535568.0   
2015-12-11 00:00:00-05:00  320730.0  48.63  49.0195  48.37  48.640  1878354.0   
2015-12-14 00:00:00-05:00  581378.0  48.57  49.3800  48.51  49.340  1679383.0   
2015-12-15 00:00:00-05:00  383071.0  49.78  50.3800  49.63  49.945  1776713.0   
2015-12-16 00:00:00-05:00  267432.0  50.20  51.2640  50.10  51.230  1884046.0   

                             PFE                                 PINS       \
                            open   high     low  close    volume open high   
2015-12-10 00:00:00-05:00  32.43  32.72  32.240  32.37  39717813 NaN  NaN    
2015-12-11 00:00:00-05:00  32.11  32.23  31.970  32.07  34679238 NaN  NaN    
2015-12-14 00:00:00-05:00  31.99  32.21  31.840  32.18  32025435 NaN  NaN    
2015-12-15 00:00:00-05:00  32.59  32.82  32.165  32.25  41728736 NaN  NaN    
2015-12-16 00:00:00-05:00  32.43  32.67  31.990  32.57  40161850 NaN  NaN    

                                           UBER                       WKHS  \
                          low close volume open high low close volume open   
2015-12-10 00:00:00-05:00 NaN NaN   NaN    NaN  NaN  NaN NaN   NaN     6.6   
2015-12-11 00:00:00-05:00 NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN    
2015-12-14 00:00:00-05:00 NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN    
2015-12-15 00:00:00-05:00 NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN    
2015-12-16 00:00:00-05:00 NaN NaN   NaN    NaN  NaN  NaN NaN   NaN    NaN    

                                                     
                          high   low close   volume  
2015-12-10 00:00:00-05:00  6.7  6.01  6.4   24236.0  
2015-12-11 00:00:00-05:00 NaN  NaN   NaN   NaN       
2015

In [37]:
# Select only close prices
stocks_close_only = pd.concat([stock_historical['KO']['close'], stock_historical['PFE']['close'], stock_historical['O']['close'], stock_historical['GILD']['close'], stock_historical['D']['close'], stock_historical['MAIN']['close'],
                     stock_historical['UBER']['close'], stock_historical['PINS']['close'], stock_historical['WKHS']['close'], stock_historical['CTSH']['close']], 
                    axis=1)
stocks_close_only.columns = ['KO', 'PFE', 'O', 'GILD', 'D', 'MAIN', 'UBER', 'PINS', 'WKHS', 'CTSH']
stocks_close_only.head()

,KO,PFE,O,GILD,D,MAIN,UBER,PINS,WKHS,CTSH
2015-12-10 00:00:00-05:00,42.74,32.37,48.950,102.63,65.20,30.9200,NaN,NaN,6.4,59.52
2015-12-11 00:00:00-05:00,42.28,32.07,48.640,100.02,64.89,30.1200,NaN,NaN,NaN,58.29
2015-12-14 00:00:00-05:00,42.43,32.18,49.340,100.50,65.42,29.0799,NaN,NaN,NaN,59.75
2015-12-15 00:00:00-05:00,43.07,32.25,49.945,102.02,66.15,29.0300,NaN,NaN,NaN,59.66
2015-12-16 00:00:00-05:00,43.83,32.57,51.230,103.32,67.57,29.2600,NaN,NaN,NaN,62.41


In [38]:
stock_returns = stocks_close_only.pct_change().dropna()
stock_returns.head()

,KO,PFE,O,GILD,D,MAIN,UBER,PINS,WKHS,CTSH
2019-05-13 00:00:00-04:00,-0.003112,-0.003192,0.003370,-0.021354,0.013814,-0.012120,-0.103490,-0.081351,0.116223,-0.038377
2019-05-14 00:00:00-04:00,0.013111,0.001478,0.007739,0.014183,-0.012965,0.013520,0.072744,0.087805,-0.073147,0.015120
2019-05-15 00:00:00-04:00,0.010682,0.012297,0.003768,0.003535,-0.004155,0.003953,0.030789,-0.011156,-0.007712,0.014808
2019-05-16 00:00:00-04:00,0.008435,0.011905,0.000577,0.007351,0.013997,-0.000492,0.042243,0.077428,-0.217617,0.000597
2019-05-17 00:00:00-04:00,-0.008163,-0.004562,-0.000577,0.008361,0.003451,-0.006893,-0.023760,-0.136045,-0.026490,0.014071


In [39]:
stock_returns.index = pd.to_datetime(stock_returns.index).date
stock_returns.head()

,KO,PFE,O,GILD,D,MAIN,UBER,PINS,WKHS,CTSH
2019-05-13,-0.003112,-0.003192,0.003370,-0.021354,0.013814,-0.012120,-0.103490,-0.081351,0.116223,-0.038377
2019-05-14,0.013111,0.001478,0.007739,0.014183,-0.012965,0.013520,0.072744,0.087805,-0.073147,0.015120
2019-05-15,0.010682,0.012297,0.003768,0.003535,-0.004155,0.003953,0.030789,-0.011156,-0.007712,0.014808
2019-05-16,0.008435,0.011905,0.000577,0.007351,0.013997,-0.000492,0.042243,0.077428,-0.217617,0.000597
2019-05-17,-0.008163,-0.004562,-0.000577,0.008361,0.003451,-0.006893,-0.023760,-0.136045,-0.026490,0.014071


# Annualized Std Dev

In [ ]:
sharpe_stocks = (stock_returns.mean()*252)/(stock_returns.std() * np.sqrt(252))
sharpe_stocks.sort_values(inplace=True)

In [ ]:
sharpe_stocks.plot(kind='bar', title='Sharpe Ratios for Stocks')

In [ ]:
# Sharpes for bonds
sharpe_bonds = (bonds_returns.mean()*252)/(bonds_returns.std()*np.sqrt(252))
sharpe_bonds.sort_values(inplace=True)

In [ ]:
sharpe_bonds.plot(kind='bar', title='Sharpe Ratios for Bonds')

In [ ]:
# Sharpe for crypto
sharpe_crypto = (crypto_pct_change.mean()*365)/(crypto_pct_change.std()*np.sqrt(365))
sharpe_crypto.sort_values(inplace=True)

In [ ]:
sharpe_crypto.plot(kind='bar', title='Sharpe Ratios for Crypto')

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2015-08-07', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2020-08-07', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# YOUR CODE HERE!

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Printing the simulation input data
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

### Calculate the expected portfolio return at the 95% lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")